<a href="https://colab.research.google.com/github/gihuni99/Basic_MLDL/blob/main/Ch05_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree(결정 트리)

예제) 도수, 당도, pH값으로 와인의 종류(레드 또는 화이트)를 구별하는 것

### Logistic Regression으로 와인 분류하기

6497개의 와인 샘플 데이터가 있다. pandas를 사용해 직접 불러오자

In [ ]:
import pandas as pd
wine=pd.read_csv('https://bit.ly/wine_csv_data')

head()로 wine dataframe의 첫 5개의 sample을 확인해보면 첫 3개의 column은 각각 'alcohol', 'sugar', 'pH'의 feature를 나타내고, 4번째 column은 class(0이면 레드, 1이면 화이트)를 나타낸다.

이는 binary classification 문제이고, 화이트 와인이 양성 클래스이다.

In [ ]:
wine.head()

logistic regression model을 training하기 전에 pandas dataframe의 유용한 method에 대해 먼저 알아보자.

* **info()** method: 각 column의 data type과 누락된 데이터가 있는지 확인하는데 유용하다.

아래 결과를 보면 6497개의 sample이 있고, 4개의 column은 모두 실수값이다. non-null을 통해 누락값도 없는 것을 확인할 수 있다.

In [ ]:
wine.info()

* **describe()** method: column에 대한 간략한 통계를 출력해준다.(최대, 최소, 평균값 등)

mean, std, min~max값을 출력해준다. 아래 출력값을 통해 각 feature들의 scale이 다름을 알 수 있다.

In [ ]:
wine.describe()

각 feature들의 scale을 맞추기 위해 scikit-learn의 StandardScaler클래스를 사용하겠다. 표준화하기 전에 dataframe을 numpy배열로 바꾸고, train set과 test set으로 나눈다.

'alcohol', 'sugar', 'pH' column을 numpy 배열로 바꾸어 data 배열에 저장하고, 'class' column은 target 배열에 저장

In [ ]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

이제 train_test_split 패키지를 사용하여 train, test set을 나눠준다.

이때 'test_size' parameter는 test set의 비율을 의미하는데 기본값은 25%, 즉 0.25이다. 하지만 sample의 개수가 충분히 많으므로 20%정도만 test set으로 지정하였다.

In [ ]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42) #test set은 전체 데이터의 20%

아래 결과를 통해 train set는 5197개의 sample, test set은 1300개의 sample을 가지고 있는 것을 알 수 있다.

In [ ]:
print(train_input.shape,test_input.shape)

train, test set을 나누었으므로, 이제 feature들을 standard score로 변환시켜 scale을 맞춰준다(StandardScaler활용)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train_input)
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input)

이제 train_scaled와 test_scaled를 사용하여 logistic regression model을 학습시켜보자.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(train_scaled,train_target)
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))

위 결과를 통해 다소 Underfitting되었다는 것을 알 수 있다. 문제를 해결하기 위해 'C'(regularization강도 파라미터) 또는 'solver'(사용할 알고리즘 선택) paramter를 바꾸거나, 다항 feature를 만들어 추가할 수도 있다.

_설명하기 쉬운 모델과 어려운 모델_

모델 설명을 위해 Logistic Regression이 training한 weight와 bias를 출력해보자

In [ ]:
print(lr.coef_,lr.intercept_)

위 결과를 보면 알코올 도수값에 0.513, 당도에 1.673, pH에 -0.688을 곱하고, bias인 1.818과 함께 더하였을 때, 값이 0보다 크면 화이트 와인, 작으면 레드와인인 것을 알 수 있고, 77%의 정확도로 classification한다는 것을 알 수 있다.

하지만, 각 weight의 수치적인 의미까지 정확하게 알 수는 없다. 다만 도수와 당도가 높을 수록 화이트 와인일 가능성이 높고, pH가 높을수록 레드와인일 확률이 높다는 것을 알 수 있을 뿐이다. 여기에 다항 feature를 추가한다면 설명하기 더 어려워진다. 따라서 모델을 설명할 새로운 방법이 필요하다.

### Decision Tree(결정 트리)

**Decision Tree** 모델은 스무고개와 같다. 데이터를 잘 나눌 수 있는 질문을 찾는다면, 질문을 추가하여 accuracy를 높일 수 있다.

Scikit-Learn은 Decision Tree 알고리즘을 제공한다.

**DecisionTreeClassifier** 클래스를 사용하여 Decision Tree 모델을 학습시켜보자.(방법은 이전 모델들과 동일하다)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42) #sklearn의 decision tree는 최적의 classification을 찾기 전 feature의 순서를 무작위로 섞는다.(random_state로 고정)
dt.fit(train_scaled,train_target)
print(dt.score(train_scaled,train_target))
print(dt.score(test_scaled,test_target))

위 결과를 보면 accuracy가 높아진 것을 볼 수 있다. 하지만 Overfitting이 발생하였다.

Scikit-Learn은 **plot_tree()**를 사용해 decision tree를 그림으로 나타내 준다. 다음 코드를 확인해보자.

결과에서 가장 위의 node를 'root node', 가장 아래 node를 'leaf node'라고 말한다.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(10,7))
plot_tree(dt)
plt.show()

위 그림이 복잡하여 **plot_tree**함수에서 tree의 depth를 제한하여 출력해보자.

* **max_depth** parameter를 1로 설정하면 root를 제외하고 하나의 node를 더 확장하여 그린다.

* **filled** parameter에서 클래스에 맞도록 node의 색을 칠할 수 있다.

* **feature_names** parameter에서 feature의 이름을 전달할 수 있다.

In [ ]:
plt.figure(figsize=(10,7))
plot_tree(dt,max_depth=1,filled=True,feature_names=['alcohol','sugar','pH'])
plt.show()

위 결과를 통해 tree가 간단하게 나타난 것을 알 수 있다. 위 node에서 suger, gini, sample, value는 다음과 같은 의미를 갖는다.

* **sugar(당도)**: 테스트 조건. 즉 만족을 하면 왼쪽 node, 만족을 하지 않으면 오른쪽 node로 분류된다.
* **gini**: 불순도
* **samples**: 해당 node의 총 sample 수
* **value**: 클래스별 샘플 수(root node에서는 음성 클래스(레드) 1258개, 양성 클래스(화이트) 3939개가 있다.)

추가적으로 **plot_tree()**함수에서 **filled=True**로 지정하면 클래스마다 색을 부여하고, 어떠한 클래스의 비율이 높아지면 점점 진하게 표시된다.(직관적)

**Decision Tree**에서의 예측 방법은 **leaf node**에서 가장 많은 클래스가 예측 클래스가 되는 것(K-NN과 매우 유사)

즉 위 예시처럼 leaf node가 2개인 상태로 학습을 종료한다면 모든 sample들이 화이트 와인으로 예측된다.(두 node 모두 양성 클래스가 많다)

_**gini(불순도)**_

**gini**는 **Gini impurity(지니 불순도)**를 의미한다. DecisionTreeClassifier 클래스의 **criterion** parameter의 기본값이 'gini'이다.

그렇다면 root node는 어떻게 당도 -0.239를 기준으로 sample들을 분할하였을까?

**gini impurity** = 1 - ($양성 클래스의 비율^2$ + $음성 클래스의 비율^2$)

위 식처럼 간단하게 나타낼 수 있다.(클래스가 더 많아도 계산 방법은 동일)

즉, 1 - ($(1258/5197)^2$ + $(3939/5197)^2$)=0.367이 나온다.

Decision Tree 모델은 parent node와 child node간의 impurity차이가 가능한 크도록 tree를 성장시킨다. 불순도 차이에 대한 계산은 다음과 같다.

부모의 불순도 - (왼쪽 노드 샘플수 / 부모의 샘플 수) x 왼쪽 노드 불순도 - 오른쪽 노드 샘플수 / 부모의 샘플 수) x 오른쪽 노드 불순도

= 0.367 - (2992/5197) x 0.481 - (2275/5197) x 0.069 = 0.066이다.

여기서 parent node와 child node의 불순도 차이를 **information garin(정보 이득)**이라고 하고, 예시의 information gain은 0.066이다.

다른 불순도 기준도 존재한다. DecisionTreeClassifier 클래스에서 **criterion='entropy'를 사용하여 **entropy impurity**를 사용할 수 있다.

gini impurity와 다르게 '제곱'이 아닌 로그2를 사용한다.

지금까지 gini impurity에 대해 알아보았다. 앞선 예시에서 tree가 제한 없이 커졌기 때문에 overfitting이 발생했다. 이를 해결해보자.

_**가지치기(Pruning)**_

decision tree에서 **Pruning**을 하지 않으면 train set에 대해 overfitting이 발생한다.(따라서 pruning 필요)

decision tree에서 pruning을 하는 가장 간단한 방법은 tree의 최대 depth를 지정하는 것이다. DecisionTreeClassifier 클래스의 **max_depth** parameter를 3으로 지정하여 모델을 만들어보자.

In [ ]:
dt=DecisionTreeClassifier(max_depth=3,random_state=42)
dt.fit(train_scaled,train_target)
print(dt.score(train_scaled,train_target))
print(dt.score(test_scaled,test_target))

위 결과를 보면 train set의 성능은 하락했고, test set의 성능은 그대로이다. 해당 tree를 직접 그려 살펴보자.

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt,filled=True,feature_names=['alcohol','sugar','pH'])
plt.show()

위 결과를 보면 depth가 1인 node는 모두 당도(sugar)를 기준으로 train set을 나눈다.(root node는 depth가 0)

가장 마지막인 depth가 3인 node가 leaf node이다. leaf node에서 왼쪽에서 3번째 node에 도달해야만 레드와인으로 예측한다.

* 당도는 -0.239보다 작고 -0.802보다 커야 한다.
* 도수는 0.454보다 작아야 한다.

위 조건을 만족해야 red wine으로 분류될 수 있다. 하지만 생각해보면 당도가 -0.239라는 것은 이상하다. 생각해보면 Decision tree는 impurity를 기준으로 sample을 분할하고, impurity는 클래스별 비율을 가지고 계산한다. 따라서 Feature의 Scale은 알고리즘에 영향을 주지 못한다.(preprocessing이 필요없다)

다음 코드로 전처리(StandardScaler())전 data로 훈련해보자.

아래 결과를 보면 결과가 같은 것을 볼 수 있다.

In [ ]:
dt=DecisionTreeClassifier(max_depth=3,random_state=42)
dt.fit(train_input,train_target)
print(dt.score(train_input,train_target))
print(dt.score(test_input,test_target))

아래 Decision Tree는 Feature의 scale이 정규화되지 않아 직관적으로 이해하기 좋다.

In [ ]:
plt.figure(figsize=(20,15))
plot_tree(dt,filled=True,feature_names=['alcohol','sugar','pH'])
plt.show()

마지막으로 Decision Tree는 어떤 Feature가 가장 유용한지 나타내는 feature의 중요도를 계산해준다. 결과를 확인하기 전, depth가 1일 때 'sugar'를 사용했기 때문에 'sugar' feature의 중요도가 가장 높을 것으로 예상된다.

feature의 중요도는 decision tree model의 **feature_importances_** attribution에 저장되어 있다.

In [ ]:
print(dt.feature_importances_)

위 결과를 통해 당도가 0.87정도로 중요도가 가장 높은 것을 볼 수 있다.

machine learning모델은 **'Black Box'**라는 말이 있다. 실제로 모델의 weight나 bias가 왜 그렇게 학습되는지 설명하기 어렵기 때문이다. 여기서 Decision Tree는 직관적인 모델을 만든다.(더 활용되는 사례는 뒷 절에서 배워보자)

### 추가공부

_**pandas**_
* **info()**: dataframe의 요약된 정보를 출력해준다. **index**, **column type**, **null이 아닌 값의 개수**, 그리고 **메모리 사용량**을 제공한다. **vervbose** parameter의 기본값(True)를 False로 지정하면, 각 column의 정보를 출력하지 않는다.
* **describe()**: dataframe의 column 통계값을 제공. data가 수치일 경우 최소, 최대, 평균, 표준편차, 사분위값 등을 출력. data가 문자열과 같은 객체 타입의 column은 가장 자주 등장하는 값과 횟수 등이 출력. **percentiles** parameter는 백분위수를 지정(기본값은 [0.25,0.5,0.75])

_**Scikit-Learn**_
* **DecisionTreeClassifier**: Decision Tree Classification모델의 클래스. **criterion** paramter는 불순도(impurity)를 지정하며 'gini'혹은 'entropy'로 지정 가능(기본값은 'gini'), **splitter** parameter는 node를 분할하는 전략을 선택(기본값은 'best'로 information gain이 최대가 되도록 분할, 'random'이면 임의로 분할), **max_depth**는 tree의 depth를 지정(기본값은 None으로 leaf node가 pure하거나, min_samples_split보다 sample 개수가 적을때까지 tree가 커진다.), **min_samples_split**은 node를 나누기 위한 최소 샘플 개수(기본값은 2), **max_features**는 최적의 분할을 위해 탐색할 feature의 개수를 지정(기본값은 None으로 모든 feature를 사용)
* **plot_tree()**: Decision Tree 모델을 시각화. 첫번째 parameter로 DecisionTreeClassifier의 객체를 전다, **max_depth** parameter로 tree의 depth지정(기본값은 Node로 모든 node출력), **feature_names** parameter로 feature의 이름을 지정, **filled** parameter를 True로 지정하면 target값에 따라 node의 색 채움